In [6]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import urllib.request
import urllib.parse
import json
import random

In [7]:
tree = ET.parse('data/volnamista/vm20151014.xml')
root = tree.getroot()

In [8]:
out = {}

In [9]:
for misto in root:
    uid = misto.attrib['uid']
    celkemVm = misto.attrib['celkemVm']
    out[uid] = {'celkemVm': celkemVm}
    for val in misto:
        tag = str(val.tag).split('}')[1]
        for attr in val.attrib:
            out[uid][tag + '_' + attr] =  val.attrib[attr]

In [10]:
outFrame = pd.DataFrame.from_dict(out, orient='index')

In [11]:
outFrame.columns

Index(['ZELENE_KARTY_vmRezervProZk', 'MZDA_min',
       'MODRE_KARTY_vmRezervProVyhovMk', 'PRACPRAVNI_VZTAH_nazev',
       'PRACOVISTE_okresKod', 'ZELENE_KARTY_vmRezervProVydanZk', 'FIRMA_ic',
       'MODRE_KARTY_celkemVmProMk', 'OBOR_nazev', 'PRACOVISTE_cobce',
       'KONOS_jmeno', 'ZELENE_KARTY_celkemVmProZk',
       'ZELENE_KARTY_pocetVmProZk', 'VHODNE_PRO_cizince',
       'MODRE_KARTY_pocetVmProMk', 'SMENNOST_nazev', 'PRACOVISTE_obec',
       'ZAMEST_KARTY_vmRezervProVyhovZm', 'MZDA_max',
       'ZAMEST_KARTY_vmRezervProZm', 'celkemVm', 'VHODNE_PRO_absolventyVs',
       'PRACOVISTE_cp', 'ZELENE_KARTY_vmRezervProVyhovZk',
       'MODRE_KARTY_vmRezervProMk', 'PRACOVISTE_ulice',
       'MODRE_KARTY_vmRezervProVydanMk', 'ZELENE_KARTY_klicovyPersonal',
       'VHODNE_PRO_absolventySs', 'ZAMEST_KARTY_celkemVmProZm',
       'PRACPRAVNI_VZTAH_kod', 'KONOS_email', 'PROFESE_kod',
       'ZELENE_KARTY_typZk', 'MIN_VZDELANI_kod', 'MIN_VZDELANI_nazev',
       'URAD_PRACE_nazev', 'SMENNOST_kod'

In [12]:
outFrame.OBOR_nazev.value_counts()

Výroba a provoz            15700
Obchod a cestovní ruch      9507
Služby                      4381
Doprava                     2962
Stavebnictví                2751
Zdravotnictví               2256
Obrana a ochrana            1828
Finance                     1231
Administrativa              1135
Informační technologie       953
Zemědělství a lesnictví      867
Výchova a vzdělávání         847
Management                   217
Kultura a sport              172
Právo                         68
Věda a výzkum                 53
dtype: int64

In [13]:
def addrjoin(ul, cp, ob, psc):
    if (ul == ul):
        return ul + ' ' + str(cp) + ', '+ ob + ' ' + str(psc).replace(' ', '')
    else:
        return str(cp) + ', ' + ob + ' ' + str(psc).replace(' ', '')

In [14]:
outFrame['addr'] = outFrame.apply(lambda row: addrjoin(row['PRACOVISTE_ulice'], row['PRACOVISTE_cp'], row['PRACOVISTE_obec'], row['PRACOVISTE_psc']), axis=1)

In [15]:
outFrame

,ZELENE_KARTY_vmRezervProZk,MZDA_min,MODRE_KARTY_vmRezervProVyhovMk,PRACPRAVNI_VZTAH_nazev,PRACOVISTE_okresKod,ZELENE_KARTY_vmRezervProVydanZk,FIRMA_ic,MODRE_KARTY_celkemVmProMk,OBOR_nazev,PRACOVISTE_cobce,...,VZDELANI_nazev,DOVEDNOST_popis,POVOLANI_kod,VZDELANI_kod,DOVEDNOST_nazev,DOVEDNOST_kod,POVOLANI_nazev,POVOLANI_praxe,DOVEDNOST_praxe,addr
1006970775,0,18000,0,Pracovní poměr,3302,0,26089505,0,Služby,Dolní Třebonín,...,NaN,NaN,NaN,NaN,Řidičský průkaz - řízení osobních automobilů -...,91,NaN,NaN,NaN,"196, Dolní Třebonín 38201"
1059990781,0,22000,0,Pracovní poměr,3404,0,27905918,0,Zdravotnictví,Nýrsko,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Šmilovského 640, Nýrsko 34022"
1070940723,0,80,0,Pracovní poměr,3702,0,26271648,0,Výroba a provoz,Lesná,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Okružní 813, Brno 63800"
110870710,0,9200,0,Pracovní poměr,3202,0,25740172,0,Výroba a provoz,Žebrák,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pivovarská 178, Žebrák 26753"
1110840703,0,18000,0,Pracovní poměr,3609,0,11107669,0,Výroba a provoz,Dolní Předměstí,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Starohradská 569, Polička 57201"
111240765,0,15000,0,Pracovní poměr,3100,0,26110709,0,Doprava,Horní Počernice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sezemická 2853, Praha 19300"
113140704,0,18000,0,Pracovní poměr,3410,0,68779020,0,Obchod a cestovní ruch,Bor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"náměstí Republiky 286, Bor 34802"
116050704,0,12000,0,Dohoda o provedení práce,3609,0,74608118,0,Obchod a cestovní ruch,Lány,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kapitána Jaroše 59, Svitavy 56802"
116510775,0,12400,0,Pracovní poměr,3610,0,73585548,0,Obchod a cestovní ruch,Dvůr Králové nad Labem,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Purkyňova 3007, Dvůr Králové nad Labem 54401"
1178920735,0,9200,0,Pracovní poměr,3202,0,64942457,0,Výroba a provoz,Hořovice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Na Cintlovce 1468, Hořovice 26801"


In [16]:
outFrame.to_csv(r'data/mista_2015_10_14.csv', encoding='utf-8')